### Q1. Enhanced Maze Navigation with Multiple Goals
● Description: Modify the given Best-First Search to find a path through a maze
with multiple goal points. The algorithm should visit all goal points and return
the shortest path covering all goals.

● Challenge: The maze will have several dead ends and multiple goal points at
different locations.

In [ ]:
from queue import PriorityQueue

class Node:
    def __init__(self, position, parent=None, cost=0, goals_left=None):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.goals_left = goals_left if goals_left else set()

    def __lt__(self, other):
        return self.cost < other.cost

def heuristic(current_pos, goals_left):
    if not goals_left:
        return 0
    return min(abs(current_pos[0] - g[0]) + abs(current_pos[1] - g[1]) for g in goals_left)

def reconstruct_path(node):
    path = []
    while node:
        path.append(node.position)
        node = node.parent
    return path[::-1]

def best_first_search_multiple_goals(maze, start, goal_positions):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start, None, 0, set(goal_positions))
    frontier = PriorityQueue()
    frontier.put((0, start_node))
    visited = set()

    while not frontier.empty():
        _, current_node = frontier.get()
        current_pos = current_node.position
        goals_left = current_node.goals_left.copy()

        if current_pos in goals_left:
            goals_left.remove(current_pos)

        if not goals_left:
            return reconstruct_path(current_node)

        visited.add((current_pos, frozenset(goals_left)))

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)

            if (0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and
                maze[new_pos[0]][new_pos[1]] == 0):

                new_node = Node(new_pos, current_node, current_node.cost + 1, goals_left)

                if (new_pos, frozenset(goals_left)) not in visited:
                    frontier.put((heuristic(new_pos, goals_left) + new_node.cost, new_node))

    return None

maze = [
    [0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start = (0, 0)
goal_positions = [(4, 4), (2, 3)]

path = best_first_search_multiple_goals(maze, start, goal_positions)
if path:
    print("Shortest Path covering all goals:", path)
else:
    print("No path found")


No path found


### Q2. Implement an A* Search where the edge costs change dynamically at random intervals.
The algorithm should adapt to these changes and always find the
optimal path. Recompute and adjust paths in real time without restarting the
algorithm from scratch.

In [5]:
from queue import PriorityQueue
import random

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def dynamic_a_star(maze, start, goals):
    rows, cols = len(maze), len(maze[0])
    frontier = PriorityQueue()
    frontier.put((0, start, []))
    visited = set()
    costs = {start: 0}
    completed_goals = set()
    full_path = []

    while not frontier.empty():
        curr_cost, curr_pos, curr_path = frontier.get()

        if curr_pos in visited:
            continue

        visited.add(curr_pos)
        curr_path.append(curr_pos)

        if curr_pos in goals:
            completed_goals.add(curr_pos)
            full_path.extend(curr_path)
            if completed_goals == goals:
                return full_path
            curr_path = [curr_pos]

        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            new_pos = (curr_pos[0] + dx, curr_pos[1] + dy)
            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0:
                new_cost = curr_cost + random.randint(1, 10)
                if new_pos not in costs or new_cost < costs[new_pos]:
                    costs[new_pos] = new_cost
                    remaining_goals = goals - completed_goals
                    if remaining_goals:
                        priority = new_cost + min(heuristic(new_pos, goal) for goal in remaining_goals)
                    else:
                        priority = new_cost
                    frontier.put((priority, new_pos, curr_path.copy()))

    return "No path to all goals"

maze = [
    [0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start = (0, 0)
goals = {(4, 4), (0, 3), (3, 1)}

path_dynamic_astar = dynamic_a_star(maze, start, goals)

print("\nDynamic A* Optimized Path")
if isinstance(path_dynamic_astar, list):
    for step in path_dynamic_astar:
        print(f"→ {step}")
else:
    print(path_dynamic_astar)



Dynamic A* Optimized Path
No path to all goals


### Q3. Delivery Route Optimization with Time Windows
● **Description**: Using the Greedy Best-First Search, optimize delivery routes for a set of delivery points. Each delivery point has a specific time window for delivery,
and the algorithm must prioritize those with stricter deadlines.

● **Challenge:** Ensure that the algorithm handles time constraints efficiently while
minimizing total travel distance.

In [1]:
from queue import PriorityQueue

def greedy_best_first_delivery(graph, start, deliveries):
    pq = PriorityQueue()
    pq.put((0, start, []))
    visited = set()
    completed_deliveries = set()
    optimized_path = []

    while not pq.empty():
        cost, curr, curr_path = pq.get()

        if curr in visited:
            continue
        visited.add(curr)

        curr_path.append(curr)

        if curr in deliveries:
            completed_deliveries.add(curr)
            optimized_path.extend(curr_path)
            curr_path = []

        if completed_deliveries == deliveries:
            return optimized_path

        for neighbor, travel_cost in graph.get(curr, []):
            if neighbor not in visited:
                pq.put((travel_cost, neighbor, curr_path.copy()))

    return "Not all deliveries are possible."

delivery_graph = {
    'A': [('B', 3), ('C', 5)],
    'B': [('D', 2), ('E', 4)],
    'C': [('E', 1), ('F', 7)],
    'D': [('G', 6)],
    'E': [('G', 3)],
    'F': [('G', 2)],
    'G': []
}

start_location = 'A'
delivery_points = {'D', 'E', 'G'}

delivery_path = greedy_best_first_delivery(delivery_graph, start_location, delivery_points)

print("Optimized Delivery Route:", delivery_path)


Optimized Delivery Route: ['A', 'B', 'D', 'A', 'B', 'E', 'G']
